# pyident — Underactuation sweep (minimal notebook)
This notebook is a **thin wrapper** around the same core function the CLI uses:
`pyident.experiments.underactuation.sweep_underactuation`. It writes a CSV and
shows quick diagnostics/plots.

> Tip: If you open this in Colab, uncomment the first cell to install your repo.

In [ ]:
# (Colab only) Install your repo — uncomment if needed
# !pip install -U pip
# !pip install -U matplotlib pandas
# %pip install -e .  # if running within a cloned repo

In [ ]:
# Setup
import os, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# JAX env (safe defaults; overridden by your local env/CLI)
os.environ.setdefault("JAX_PLATFORM_NAME", "cpu")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")

from pyident.experiments.underactuation import sweep_underactuation

## Parameters (edit me)

In [ ]:
# --- core experiment parameters ---
n = 8
m_values = [1, 2, 3, 4]
T = 200
dt = 0.05
ensemble = "ginibre"     # or "sparse", "stable", "binary"
signal = "prbs"
sigPE = 12
seeds = [0, 1, 2]
algs = ("dmdc",)         # estimators to run

# options
x0_mode = None           # or "zero"
sparse_which = "B"       # used if ensemble == "sparse"
p_density_B = 0.1        # used if ensemble == "sparse"
use_jax = False
jax_x64 = True

# output CSV path
out_csv = "runs/underact_notebook.csv"  # directories will be created

## Run

In [ ]:
# Ensure output directory exists
import os
os.makedirs(os.path.dirname(out_csv) or ".", exist_ok=True)

# Run the exact same core routine as the CLI
sweep_underactuation(
    n=n,
    m_values=m_values,
    T=T,
    dt=dt,
    ensemble=ensemble,
    signal=signal,
    sigPE=sigPE,
    seeds=seeds,
    algs=algs,
    out_csv=out_csv,
    use_jax=use_jax,
    jax_x64=jax_x64,
    x0_mode=x0_mode,
    sparse_which=sparse_which,
    p_density_B=p_density_B,
)

print(f"Wrote {out_csv}")

## Quick diagnostics

In [ ]:
import pandas as pd, numpy as np
df = pd.read_csv(out_csv)

# E[K_rank] by m
gK = df.groupby("m")["K_rank"].mean()
print("E[K_rank] by m:\n", gK)

# Spearman(m, A_err) (if estimator column present)
if "est.dmdc.A_err_PV" in df.columns:
    gerr = df.groupby("m")["est.dmdc.A_err_PV"].mean().reset_index()
    rho = gerr["m"].corr(gerr["est.dmdc.A_err_PV"], method="spearman")
    print(f"Spearman(m, A_err): {rho:.3f}")

## Plots

In [ ]:
# Plot 1: E[V_dim] vs m (mean ± std)
if "V_dim" in df.columns:
    mu = df.groupby("m")["V_dim"].mean()
    sd = df.groupby("m")["V_dim"].std()

    fig, ax = plt.subplots(figsize=(5, 3.2))
    ax.errorbar(mu.index.values, mu.values, yerr=sd.values, lw=2, capsize=3)
    ax.set_xlabel("m")
    ax.set_ylabel("E[V_dim]")
    ax.set_title("Underactuation: visible subspace vs m")
    ax.grid(True, alpha=0.3)
    plt.show()

In [ ]:
# Plot 2: DMDc success rate by m (threshold based)
if "est.dmdc.A_err_PV" in df.columns:
    tau = max(1e-8, df["est.dmdc.A_err_PV"].median() * 10.0)
    succ = (df["est.dmdc.A_err_PV"] < tau).groupby(df["m"]).mean().reset_index()

    fig, ax = plt.subplots(figsize=(5, 3.2))
    ax.plot(succ["m"], succ["est.dmdc.A_err_PV"], marker="o", lw=2)
    ax.set_xlabel("m")
    ax.set_ylabel("success rate (A_err<thr)")
    ax.set_title("DMDc success vs underactuation level")
    ax.grid(True, alpha=0.3)
    plt.show()